In [ ]:
!pip install datasets
!pip install torch==2.0.0 torchtext==0.15.1

In [ ]:
!pip install -U datasets

In [ ]:
# Імпорт необхідних бібліотек
import datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import time

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.calibration import calibration_curve


# Встановлення seed для відтворюваності результатів
SEED = 2
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Перевірка доступності CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Технологія, яка використовується: {device}")

# Завантаження даних IMDb
dataset = load_dataset('imdb')

print(dataset)

# Розділення на тренувальний та тестувальний набори даних
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])
train_data_df, test_data_df = dataset['train'].to_pandas(), dataset['test'].to_pandas()

# Вивід перших та останніх 3 коментарів (тренувальний датафрейм)
subset = pd.concat([train_data_df.head(3), train_data_df.tail(3)])
print(subset)

# Заміна числових міток на текстові
train_data_df['label'] = train_data_df['label'].map({0: 'negative', 1: 'positive'})
test_data_df['label'] = test_data_df['label'].map({0: 'negative', 1: 'positive'})

# Побудова графіків для огляду тренувального та тестувального наборів
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
train_data_df['label'].value_counts().plot(kind='bar', color=['#00441b', '#207d3e'], ax=axes[0])
test_data_df['label'].value_counts().plot(kind='bar', color=['#00441b', '#207d3e'], ax=axes[1])

axes[0].set_xlabel('Sentiment')
axes[0].set_ylabel('Count')
axes[0].set_title('IMDb Train Dataset Reviews')
axes[0].set_xticklabels(['negative', 'positive'], rotation=0)

axes[1].set_xlabel('Sentiment')
axes[1].set_ylabel('Count')
axes[1].set_title('IMDb Test Dataset Reviews')
axes[1].set_xticklabels(['negative', 'positive'], rotation=0)

plt.tight_layout()
plt.show()

# Токенізація тексту
tokenizer = get_tokenizer('basic_english')
sample_text = train_data[0]['text']
sample_tokens = tokenizer(sample_text)

# Приклад токенізації
print("Приклад токенізації:")
print(f"\nТекст: {sample_text[:100]}...")
print(f"Токени: {sample_tokens[:10]}...")

# Функція для токенізації
def tokenize(text, tokenizer, max_length):
    tokens = tokenizer(text["text"])[:max_length]
    return {"tokens": tokens}

    # Токенізація датасетів
max_length = 300

train_data = train_data.map(
    tokenize, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)
test_data = test_data.map(
    tokenize, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)

# Розбиття на тренувальний та валідаційний набори
split_data = train_data.train_test_split(test_size=0.2)
train_set = split_data["train"]
validation_set = split_data["test"]

# Побудова словника
vocab = torchtext.vocab.build_vocab_from_iterator(
    train_set["tokens"],
    specials=['<unk>', '<pad>'],
    min_freq=5
)
vocab.set_default_index(vocab['<unk>'])
print(f"\nРозмір словника: {len(vocab)} токенів")

# Функція для числового представлення тексту
def numericalize_tokens(tokens, vocab):
    ids = vocab.lookup_indices(tokens["tokens"])
    return {"ids": ids}

# Перетворення токенів у числові індекси
train_set = train_set.map(numericalize_tokens, fn_kwargs={"vocab": vocab})
validation_set = validation_set.map(numericalize_tokens, fn_kwargs={"vocab": vocab})
test_data = test_data.map(numericalize_tokens, fn_kwargs={"vocab": vocab})

# Зміна формату даних на тензори
train_set = train_set.with_format(type="torch", columns=["ids", "label"])
validation_set = validation_set.with_format(type="torch", columns=["ids", "label"])
test_data = test_data.with_format(type="torch", columns=["ids", "label"])

# Функція для обробки батчів
def collate_batch(batch):
    texts, labels = [], []
    for sample in batch:
        ids = sample['ids']
        texts.append(torch.tensor(ids, dtype=torch.int64))
        labels.append(sample['label'])
    texts_padded = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=vocab['<pad>'])
    labels = torch.tensor(labels, dtype=torch.long)
    return texts_padded, labels

# Створення DataLoader
batch_size = 512
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(validation_set, batch_size=batch_size, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_batch)

# Визначення моделі CBoW
class CBoW_Model(nn.Module):
    def __init__(self, vocabulary_size, embedding_dimension, output_dimension, padding_idx):
        super().__init__()
        self.word_embeddings = nn.Embedding(vocabulary_size, embedding_dimension, padding_idx=padding_idx)
        self.fully_connected = nn.Linear(embedding_dimension, output_dimension)

    def forward(self, text):
        embedded_words  = self.word_embeddings(text)
        pooled_embedding = embedded_words.mean(dim=1)
        prediction = self.fully_connected(pooled_embedding)
        return prediction

# Параметри моделі
vocabulary_size = len(vocab)
embedding_dimension = 300
output_dimension = 2
padding_idx = vocab['<pad>']
model = CBoW_Model(vocabulary_size, embedding_dimension, output_dimension, padding_idx)

# Створення оптимізатора для навчання моделі та визначення функції втрат
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# Переміщення моделі на GPU, якщо доступно
model = model.to(device)
criterion = criterion.to(device)

# Функція для тренування моделі
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    for texts, labels in tqdm(iterator, desc='Тренування'):
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        acc = (predictions.argmax(dim=1) == labels).float().mean()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Функція для оцінки моделі
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for texts, labels in tqdm(iterator, desc='Оцінка'):
            texts, labels = texts.to(device), labels.to(device)
            predictions = model(texts)
            loss = criterion(predictions, labels)
            acc = (predictions.argmax(dim=1) == labels).float().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Тренування моделі
n_epochs = 10
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []

# Початок вимірювання часу
start_time = time.time()

for epoch in range(n_epochs):
    tl, ta = train(model, train_loader, optimizer, criterion)
    vl, va = evaluate(model, valid_loader, criterion)
    train_loss.append(tl)
    valid_loss.append(vl)
    train_acc.append(ta)
    valid_acc.append(va)
    print(f'\nЕпоха: {epoch}')
    print(f'Train loss: {tl:.3f}, Train accuracy: {ta:.3f}')
    print(f'Valid loss: {vl:.3f}, Valid accuracy: {va:.3f}')

# Кінець вимірювання часу
end_time = time.time()
training_time = end_time - start_time

# Виведення часу навчання
hours = int(training_time // 3600)
minutes = int((training_time % 3600) // 60)
seconds = int(training_time % 60)
print(f"\nЧас навчання моделі: {hours} годин, {minutes} хвилин, {seconds} секунд")

# Візуалізація результатів втрат
plt.figure(figsize=(10,6))
plt.plot(train_loss, label='Train')
plt.plot(valid_loss, label='Valid')

ax = plt.gca()
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_xticks(range(n_epochs))
ax.set_title('Loss')

plt.legend()
plt.grid()
plt.show()

# Візуалізація результатів точності
plt.figure(figsize=(10,6))
plt.plot(train_acc, label='Train')
plt.plot(valid_acc, label='Valid')

ax = plt.gca()
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_xticks(range(n_epochs))
ax.set_title('Accuracy')

plt.legend()
plt.grid()
plt.show()

# Отримання передбачень на тестовому наборі для оцінки якості моделі
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for texts, labels in tqdm(test_loader, desc='Тестування'):
        texts, labels = texts.to(device), labels.to(device)
        predictions = model(texts).argmax(dim=1)
        all_preds.extend(predictions.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

# Вивід метрик
report = classification_report(all_labels, all_preds, target_names=['neg', 'pos'], digits=3)
print(report)

# Функція для класифікації тексту
def classify_sentiment(text, model, tokenizer, vocab, device):
    model.eval()
    tokens = tokenizer(text)
    ids = vocab.lookup_indices(tokens)
    tensor = torch.tensor(ids, dtype=torch.int64).unsqueeze(0).to(device)
    prediction = model(tensor).squeeze(0)
    probability = torch.softmax(prediction, dim=-1)
    predicted_class = prediction.argmax(dim=-1).item()
    predicted_probability = probability[predicted_class].item()
    prob_positive = probability[1].item()
    return predicted_class, predicted_probability, prob_positive

# Тестування моделі на власних прикладах
sample_comments = [
    ("I absolutely loved this movie! It was fantastic!", 1),
    ("The worst movie I have ever seen. A total waste of time.", 0),
    ("This serial is not terrible, it's great", 1),
    ("Brilliant performance by the lead actor!", 1),
    ("I wouldn't recommend this film to anyone.", 0),
    ("This was a very emotional and touching story.", 1),
    ("This movie is not great, it's terrible!", 0),
    ("One of the best films I've seen this year!", 1),
    ("Mediocre storyline, but the cinematography was beautiful.", 1),
    ("I regret watching this movie. What a disaster!", 0),

    ("Deeply impressed by the brilliant cast and stunning cinematography. The plot keeps you in suspense until the end. A masterpiece worth watching again.", 1),
    ("Terrible script, awful acting, and student-level effects. The predictable plot was obvious within minutes. A complete waste of time.", 0),
    ("A vast improvement over previous seasons. Deeper characters and clearer motivations make each episode compelling. Exceeds all expectations.", 1),
    ("Phenomenal lead performance with incredible emotional range and natural expressions. Award-worthy acting that makes the film memorable.", 1),
    ("Deceptive advertising hides a film full of plot holes and flat jokes. Even a talented cast can't save this mess. Money and time wasted.", 0),
    ("Deeply touching story with universal appeal. The emotional reconciliation scene and perfect musical accompaniment create an unforgettable atmosphere.", 1),
    ("Excellent graphics wasted on a disastrous plot with illogical characters and predictable twists. The biggest disappointment of the year.", 0),
    ("A massive budget wasted on a film with great visuals but terrible plot, illogical characters, and predictable twists. The biggest disappointment of the year.", 0),
    ("Despite the hype, the film is boring with weak plot and characters. Only the cinematography is decent, but it's not enough to save this disappointment.", 0),
    ("Captivating from the first scene with remarkable attention to detail and a story that keeps you guessing. The emotional depth and subtle humor make it memorable.", 1),
    ("High hopes dashed by a messy plot and mediocre performances. The forced dialogue makes it hard to recommend.", 0),
    ("A moving exploration of love and loss with outstanding performances and breathtaking cinematography. Highly recommended for thought-provoking cinema.", 1),
    ("Starts promising but quickly falls into clichés. The shallow characters and unexplained motivations make it disappointing.", 0),
    ("A true work of art with beautiful shots, perfect soundtrack, and an extraordinary lead performance. Heartwarming and heartbreaking.", 1),
    ("A huge letdown with slow pacing and forgettable characters. The abrupt ending makes it a waste of time.", 0),
    ("Perfectly balances humor and drama with witty script and genuine emotional moments. Makes you laugh and cry.", 1),
    ("Tries to do too much and fails at everything. The convoluted plot and weak writing make it frustrating.", 0),
    ("A powerful story with well-developed characters and timely message. The moving ending makes it a must-see.", 1),
    ("Suffers from poor pacing and weak character development. The late action is too little, too late.", 0),
    ("Surprisingly enjoyable with engaging plot and relatable characters. The satisfying ending makes it worth watching.", 1),
    ("A complete mess with incoherent story and cheap effects. Gets worse as it goes on.", 0),
    ("Beautifully explores human relationships with nuanced performances and masterful direction. Highly recommended.", 1),
    ("Terrible plot, annoying characters, and unfunny jokes. Almost walked out.", 0),
    ("Exceeds expectations with original story, great acting, and stunning visuals. Gets everything right.", 1),

    # Позитивні коментарі середньої довжини
    ("The cinematography was breathtaking, and the story kept me engaged throughout. The lead actor's performance was particularly impressive, bringing depth to every scene.", 1),
    ("A delightful surprise! The plot twists were unexpected yet satisfying, and the character development was handled with great care.", 1),
    ("The film's visual style is unique and captivating. The soundtrack perfectly complements the mood, creating an immersive experience.", 1),
    ("An emotional rollercoaster that never loses its momentum. The dialogue is sharp and meaningful, making every scene count.", 1),
    ("The attention to detail in every frame is remarkable. The story unfolds beautifully, keeping you invested until the very end.", 1),
    ("A perfect blend of action and drama. The pacing is excellent, and the character arcs are satisfying to watch unfold.", 1),
    ("The chemistry between the actors is palpable. The script is witty and engaging, making for an entertaining watch.", 1),
    ("A thought-provoking journey that stays with you. The direction is masterful, and the performances are outstanding.", 1),
    ("The film's message is powerful and timely. The execution is flawless, making it both entertaining and meaningful.", 1),
    ("A beautiful exploration of human relationships. The emotional depth is remarkable, and the ending is perfect.", 1),
    ("The film's atmosphere is incredibly immersive. The attention to period details and the authentic dialogue create a vivid world.", 1),
    ("A refreshing take on a familiar genre. The innovative storytelling approach keeps you guessing until the very end.", 1),
    ("The emotional journey is beautifully crafted. Each scene builds upon the last, creating a powerful narrative arc.", 1),
    ("The visual storytelling is masterful. Every frame tells a story, and the cinematography enhances the emotional impact.", 1),
    ("A perfect balance of action and character development. The fight scenes are choreographed brilliantly.", 1),
    ("The film's message resonates deeply. The themes of love and sacrifice are handled with remarkable sensitivity.", 1),
    ("The ensemble cast delivers outstanding performances. Each actor brings something unique to their character.", 1),
    ("The musical score elevates every scene. The composer's work perfectly captures the film's emotional core.", 1),
    ("A compelling character study that never feels slow. The protagonist's journey is both believable and engaging.", 1),
    ("The world-building is exceptional. Every detail feels carefully considered and adds to the overall experience.", 1),
    ("The dialogue crackles with energy and wit. Each conversation feels natural yet meaningful.", 1),
    ("A masterclass in tension building. The suspense is maintained throughout, leading to a satisfying climax.", 1),
    ("The film's emotional intelligence is remarkable. It handles complex themes with subtlety and grace.", 1),
    ("The production design is stunning. Each location feels lived-in and authentic to the story.", 1),
    ("A beautiful exploration of family dynamics. The relationships feel genuine and deeply moving.", 1),
    ("The film's pacing is perfect. It knows exactly when to speed up and when to slow down.", 1),
    ("The character arcs are satisfying and well-earned. Each transformation feels natural and meaningful.", 1),
    ("The film's humor is perfectly balanced with its dramatic moments. The tone shifts feel natural.", 1),
    ("A powerful examination of social issues. The message is delivered without being preachy.", 1),
    ("The film's technical aspects are flawless. Every element works together to create a cohesive whole.", 1),
    ("A touching story about friendship and loyalty. The emotional beats hit hard and feel earned.", 1),
    ("The film's visual effects are seamlessly integrated. They serve the story rather than overshadow it.", 1),
    ("A compelling mystery that keeps you guessing. The clues are cleverly placed throughout.", 1),
    ("The film's emotional resonance is powerful. It stays with you long after the credits roll.", 1),
    ("A beautiful meditation on life and death. The philosophical themes are handled with care.", 1),
    ("The film's structure is innovative and engaging. The non-linear storytelling adds depth.", 1),

    # Негативні коментарі середньої довжини
    ("The plot was confusing and poorly developed. The characters lacked depth, and the dialogue felt unnatural.", 0),
    ("Despite a promising start, the film quickly loses its way. The pacing is off, and the ending is unsatisfying.", 0),
    ("The special effects look cheap and dated. The story is predictable, and the acting is wooden.", 0),
    ("A disappointing adaptation that fails to capture the essence of the source material. The changes made were unnecessary.", 0),
    ("The film tries too hard to be clever but ends up being pretentious. The plot holes are numerous and distracting.", 0),
    ("The editing is choppy and confusing. The story jumps around without clear direction, making it hard to follow.", 0),
    ("The characters are one-dimensional and uninteresting. The dialogue is cliché-ridden and often cringeworthy.", 0),
    ("The film's pacing is painfully slow. The plot developments are predictable, and the ending is anticlimactic.", 0),
    ("The direction is amateurish, and the cinematography is uninspired. The story lacks coherence and purpose.", 0),
    ("A wasted opportunity with a talented cast. The script is weak, and the direction fails to elevate the material.", 0),
    ("The film's pacing is inconsistent, with long stretches of nothing happening followed by rushed plot developments.", 0),
    ("The dialogue feels artificial and forced, with characters speaking in ways that no real person would.", 0),
    ("The special effects look dated and unconvincing, breaking the immersion at crucial moments.", 0),
    ("The plot relies too heavily on convenient coincidences, making the story feel contrived.", 0),
    ("The character development is shallow, with motivations that change without explanation.", 0),
    ("The film's attempts at humor fall flat, with jokes that feel out of place and unfunny.", 0),
    ("The editing is choppy and confusing, making it difficult to follow the story's progression.", 0),
    ("The soundtrack is intrusive and often mismatched to the tone of the scenes.", 0),
    ("The film's message is heavy-handed and preachy, lacking subtlety in its delivery.", 0),
    ("The acting is wooden and unconvincing, with even the most emotional scenes falling flat.", 0),
    ("The plot holes are numerous and distracting, making it hard to take the story seriously.", 0),
    ("The film's tone is inconsistent, shifting awkwardly between comedy and drama.", 0),
    ("The character arcs are unsatisfying, with developments that feel unearned.", 0),
    ("The cinematography is bland and uninspired, failing to create any visual interest.", 0),
    ("The film's attempts at social commentary are shallow and poorly integrated.", 0),
    ("The pacing is painfully slow, with scenes that drag on far longer than necessary.", 0),
    ("The dialogue is filled with clichés and exposition, making it hard to stay engaged.", 0),
    ("The film's world-building is inconsistent, with rules that change when convenient.", 0),
    ("The emotional moments feel manipulative rather than earned.", 0),
    ("The film's structure is confusing, with flashbacks that add little to the story.", 0),
    ("The character relationships feel artificial and undeveloped.", 0),
    ("The film's attempts at being edgy come across as juvenile and unnecessary.", 0),
    ("The plot twists are predictable and lack impact.", 0),
    ("The film's technical aspects are amateurish, with poor sound mixing and lighting.", 0),
    ("The story lacks focus, trying to do too much and succeeding at nothing.", 0),
    ("The film's ending is abrupt and unsatisfying, leaving too many questions unanswered.", 0),
    ("The film's attempt at creating tension falls completely flat. Every 'twist' is painfully obvious from miles away.", 0),
    ("A complete waste of talented actors. The script gives them nothing to work with, and their performances suffer as a result.", 0),
    ("The film's attempts at being profound come across as pretentious and shallow. It's trying too hard to be something it's not.", 0),
    ("The action sequences are poorly choreographed and confusing. It's impossible to tell what's happening half the time.", 0),
    ("The film's pacing is a mess. It drags through the middle and rushes the ending, making the whole experience unsatisfying.", 0),
    ("The dialogue is filled with exposition and unnatural conversations. Characters explain things that should be shown, not told.", 0),
    ("The film's world-building is inconsistent and full of plot holes. Rules are established and then immediately broken.", 0),
    ("The emotional moments feel manipulative and unearned. The film tries to make you care without putting in the work.", 0),
    ("The film's attempts at humor are cringeworthy and out of place. The jokes land with a thud.", 0),
    ("The character development is non-existent. Characters make decisions that make no sense for their established personalities.", 0),
    ("The film's visual style is bland and uninspired. Every shot feels like it was composed by someone who's never seen a movie.", 0),
    ("The soundtrack is intrusive and often completely mismatched to the tone of the scenes. It's distracting rather than enhancing.", 0),
    ("The film's message is muddled and confused. It tries to say too many things and ends up saying nothing at all.", 0),
    ("The editing is choppy and disorienting. Scenes cut abruptly without any sense of flow or rhythm.", 0),
    ("The film's attempts at social commentary are shallow and poorly integrated. It feels like an afterthought rather than a core element.", 0),
    ("The plot relies on too many coincidences and contrivances. Nothing feels organic or earned.", 0),
    ("The film's structure is a mess. Flashbacks and time jumps are used without any clear purpose or impact.", 0),
    ("The character relationships feel artificial and undeveloped. There's no chemistry or believable connection between anyone.", 0),
    ("The film's attempts at being edgy come across as juvenile and unnecessary. It's trying too hard to shock without substance.", 0),
    ("The plot twists are predictable and lack impact. You can see them coming from the first act.", 0),
    ("The film's technical aspects are amateurish. Poor sound mixing, inconsistent lighting, and shaky camerawork throughout.", 0),
    ("The story lacks focus, trying to do too much and succeeding at nothing. It's a mess of half-baked ideas.", 0),
    ("The film's ending is abrupt and unsatisfying, leaving too many questions unanswered and plot threads hanging.", 0),
    ("The film's attempts at creating atmosphere fall flat. The mood is inconsistent and often at odds with the story.", 0),
    ("The character motivations are unclear and often contradictory. People do things because the plot requires it, not because it makes sense.", 0),
    ("The film's attempts at being artistic come across as pretentious and self-indulgent. It's style over substance.", 0),
    ("The pacing is glacial, with scenes that drag on far longer than necessary. It feels like watching paint dry.", 0),
    ("The dialogue is filled with clichés and exposition. Characters speak in ways that no real person ever would.", 0),
    ("The film's world-building is inconsistent, with rules that change when convenient. Nothing feels established or reliable.", 0),
    ("The emotional moments feel manipulative rather than earned. The film tries to make you feel without putting in the work.", 0),
    ("The film's structure is confusing, with flashbacks that add little to the story. It's complexity for complexity's sake.", 0),
    ("The character relationships feel artificial and undeveloped. There's no chemistry or believable connection between anyone.", 0),
    ("The film's attempts at being edgy come across as juvenile and unnecessary. It's trying too hard to shock without substance.", 0),
    ("The plot twists are predictable and lack impact. You can see them coming from the first act.", 0),
    ("The film's technical aspects are amateurish. Poor sound mixing, inconsistent lighting, and shaky camerawork throughout.", 0),
    ("The story lacks focus, trying to do too much and succeeding at nothing. It's a mess of half-baked ideas.", 0),
    ("The film's ending is abrupt and unsatisfying, leaving too many questions unanswered and plot threads hanging.", 0),
    ("A thoroughly average film that fails to leave any lasting impression. Neither good enough to recommend nor bad enough to be memorable.", 0),
    ("The film is painfully mediocre, with every aspect being just passable enough to be forgettable. A perfect example of 'meh' cinema.", 0),
    ("This is the definition of a middling film - not terrible enough to be entertaining in its badness, not good enough to be worth watching.", 0),
    ("A run-of-the-mill production that brings nothing new to the table. Every element is just adequate enough to be disappointing.", 0),
    ("The film is unremarkable in every way - a perfect example of forgettable, middle-of-the-road filmmaking that wastes your time.", 0),
    ("A thoroughly undistinguished film that fails to rise above mediocrity. Every aspect is just competent enough to be frustrating.", 0),

    # Негативні неоднозначні коментарі
    ("The cinematography is stunning, but it's wasted on a story that goes nowhere. Beautiful to look at, but ultimately empty.", 0),
    ("The lead actor gives a powerful performance, but the weak script and poor direction undermine their efforts completely.", 0),
    ("The film has moments of brilliance, but they're buried under layers of unnecessary subplots and poor pacing.", 0),
    ("The production design is impressive, but it can't compensate for the lackluster storytelling and wooden performances.", 0),
    ("The soundtrack is beautiful, but it feels disconnected from the film's tone and often works against the scenes.", 0),
    ("The concept is intriguing, but the execution is so muddled that it becomes frustrating to watch.", 0),
    ("The special effects are technically impressive, but they're used in service of a story that doesn't deserve them.", 0),
    ("The film starts strong with an engaging premise, but quickly devolves into a mess of clichés and poor decisions.", 0),
    ("The character designs are creative, but the characters themselves are shallow and uninteresting.", 0),
    ("The action sequences are well-choreographed, but they're edited so poorly that it's hard to appreciate them.", 0),
    ("The film's message is important, but it's delivered with such heavy-handedness that it loses its impact.", 0),
    ("The world-building is detailed and fascinating, but the story that takes place in it is disappointingly generic.", 0),
    ("The dialogue has moments of wit, but they're overshadowed by awkward exposition and unnatural conversations.", 0),
    ("The film's atmosphere is effectively moody, but it's at odds with the inconsistent tone of the story.", 0),
    ("The supporting cast is excellent, but they're given so little to do that their talents are wasted.", 0),
    ("The film's themes are thought-provoking, but they're explored in such a superficial way that they feel wasted.", 0),
    ("The visual style is unique and striking, but it's not enough to carry the film's weak narrative.", 0),
    ("The film has a strong emotional core, but it's buried under layers of unnecessary complexity.", 0),
    ("The opening sequence is masterfully crafted, but the rest of the film fails to live up to its promise.", 0),
    ("The film's ambition is admirable, but its execution falls far short of its aspirations.", 0),
    ("The character arcs are well-conceived, but they're rushed and underdeveloped in the final product.", 0),
    ("The film's visual effects are groundbreaking, but they're used to mask a story that's paper-thin and derivative.", 0),
    ("While the lead actor brings their A-game, the rest of the cast seems to be in a completely different movie.", 0),
    ("The film's ambition is commendable, but its execution is so flawed that it becomes frustrating to watch.", 0),
    ("The cinematography is beautiful, but it's in service of a story that doesn't deserve such visual treatment.", 0),
    ("The film has a strong first act, but it completely falls apart in the second half, becoming a mess of plot holes.", 0),
    ("The production design is meticulous, but it can't make up for the lack of substance in the storytelling.", 0),
    ("The soundtrack is hauntingly beautiful, but it's often used to manipulate emotions rather than enhance them.", 0),
    ("The film's concept is innovative, but it's executed with such heavy-handedness that it loses its impact.", 0),
    ("The action sequences are technically impressive, but they're so over-edited that they become disorienting.", 0),
    ("The film's message is timely and important, but it's delivered with such obviousness that it feels patronizing.", 0),
    ("The world-building is detailed and fascinating, but the characters that inhabit it are disappointingly one-dimensional.", 0),
    ("The dialogue has moments of brilliance, but they're surrounded by clunky exposition and unnatural conversations.", 0),
    ("The film's atmosphere is effectively moody, but it's inconsistent with the tone of the story being told.", 0),
    ("The supporting cast is talented, but they're given such thin material that their performances feel wasted.", 0),
    ("The film's themes are thought-provoking, but they're explored in such a superficial way that they feel wasted.", 0),
    ("The visual style is unique and striking, but it's not enough to carry the film's weak narrative structure.", 0),
    ("The film has a strong emotional core, but it's buried under unnecessary subplots and poor pacing.", 0),
    ("The opening sequence is masterfully crafted, but the rest of the film fails to maintain that level of quality.", 0),
    ("The film's ambition is admirable, but its execution falls far short of its potential.", 0),
    ("The character development is well-conceived, but it's rushed and underdeveloped in the final product.", 0),
    ("The film's technical aspects are impressive, but they're used to cover up fundamental storytelling problems.", 0),
    ("While the direction shows promise, the weak script and poor editing undermine any potential impact.", 0),
    ("The film's pacing is well-structured initially, but it completely loses its rhythm in the final act.", 0),
    ("The production values are high, but they're wasted on a story that doesn't know what it wants to be.", 0),
    ("The film's emotional moments are well-acted, but they feel unearned due to poor character development.", 0),
    ("The visual effects are state-of-the-art, but they're used to distract from the film's narrative shortcomings.", 0),
    ("The film's premise is intriguing, but it's executed with such inconsistency that it becomes frustrating.", 0),
    ("The action choreography is impressive, but it's ruined by poor editing and excessive use of close-ups.", 0),
    ("The film's social commentary is relevant, but it's delivered with such heavy-handedness that it loses impact.", 0),
    ("The world-building is creative, but the story that takes place in it is disappointingly conventional.", 0),
    ("The dialogue has clever moments, but they're overshadowed by awkward exposition and forced humor.", 0),
    ("The film's tone is effectively dark, but it's inconsistent with the story's emotional beats.", 0),

    # Позитивні неоднозначні коментарі
    ("Despite some pacing issues in the middle, the film's powerful performances and stunning visuals make it a must-watch.", 1),
    ("While the plot has some predictable moments, the film's emotional depth and character development more than make up for it.", 1),
    ("The film's ambitious scope occasionally gets the better of it, but the sheer audacity and technical achievement are impressive.", 1),
    ("Though the dialogue can be a bit heavy-handed at times, the film's visual storytelling and emotional impact are undeniable.", 1),
    ("The film takes some time to find its footing, but once it does, it delivers a powerful and moving experience.", 1),
    ("While not every subplot lands perfectly, the main story arc is so compelling that it carries the entire film.", 1),
    ("The film's experimental structure might not work for everyone, but it's a bold and refreshing approach to storytelling.", 1),
    ("Despite some uneven pacing, the film's stunning cinematography and powerful performances make it worth watching.", 1),
    ("The film's ambitious themes are sometimes oversimplified, but the emotional core remains strong and affecting.", 1),
    ("While the special effects occasionally show their seams, the film's heart and soul shine through brilliantly.", 1),
    ("The film's unconventional narrative structure might be challenging, but it rewards those willing to engage with it.", 1),
    ("Though the film's message is delivered with some heavy-handedness, its emotional impact is undeniable.", 1),
    ("The film's slow burn approach might test some viewers' patience, but the payoff is absolutely worth it.", 1),
    ("While the film's world-building could be more detailed, the character development and emotional arcs are masterful.", 1),
    ("The film's experimental approach to storytelling might not be for everyone, but it's a refreshing change of pace.", 1),
    ("Despite some predictable plot points, the film's emotional depth and character work are exceptional.", 1),
    ("The film's ambitious scope occasionally leads to some pacing issues, but the overall vision is impressive.", 1),
    ("While the film's social commentary is sometimes obvious, its heart and humanity shine through brilliantly.", 1),
    ("The film's unconventional structure might be challenging, but it's a bold and refreshing approach to the genre.", 1),
    ("Though the film's message is delivered with some heavy-handedness, its emotional core remains powerful.", 1),
    ("The film's slow build might test some viewers' patience, but the emotional payoff is absolutely worth it.", 1),
    ("While the film's world-building could be more detailed, the character development is exceptional.", 1),
    ("The film's experimental approach might not be for everyone, but it's a refreshing change from the usual formula.", 1),
    ("Despite some predictable moments, the film's emotional depth and character work are outstanding.", 1),
    ("The film's ambitious scope occasionally leads to some pacing issues, but the overall vision is remarkable.", 1),
    ("While the film's social commentary is sometimes obvious, its heart and humanity are undeniable.", 1),
    ("The film's unconventional structure might be challenging, but it's a bold and innovative approach.", 1),
    ("Though the film's message is delivered with some heavy-handedness, its emotional impact is powerful.", 1),
    ("The film's slow burn approach might test some viewers' patience, but the payoff is incredibly rewarding.", 1),
    ("While the film's world-building could be more detailed, the character arcs are beautifully developed.", 1),
    ("The film's experimental style might not be for everyone, but it's a refreshing departure from convention.", 1),
    ("Despite some predictable elements, the film's emotional depth and character work are exceptional.", 1),
    ("The film's ambitious scope occasionally leads to some pacing issues, but the overall vision is impressive.", 1),
    ("While the film's social commentary is sometimes obvious, its heart and humanity shine through.", 1),
    ("The film's nonlinear storytelling might confuse some viewers, but it creates a uniquely engaging experience that rewards attention.", 1),
    ("Though the film's runtime is quite long, every minute is filled with meaningful character development and stunning visuals.", 1),
    ("While the film's genre-blending approach might be unconventional, it results in a refreshingly original cinematic experience.", 1),
    ("The film's complex themes might require multiple viewings to fully appreciate, but the emotional journey is deeply rewarding.", 1),
    ("Despite some initial pacing issues, the film's second half delivers such powerful emotional moments that it's impossible to forget.", 1),
    ("The film's ambitious visual style might be polarizing, but it creates a unique and memorable viewing experience.", 1),
    ("While the film's dialogue occasionally veers into philosophical territory, it adds depth to the character's motivations.", 1),
    ("The film's experimental sound design might be challenging at first, but it perfectly enhances the emotional impact of key scenes.", 1),
    ("Though the film's world-building is deliberately ambiguous, it creates an atmosphere of mystery that's utterly captivating.", 1),
    ("The film's unconventional editing style might take some getting used to, but it creates a rhythm that's uniquely its own.", 1),
    ("While the film's color palette is intentionally muted, it serves to highlight the emotional intensity of key moments.", 1),
    ("The film's complex narrative structure might require careful attention, but it rewards viewers with a deeply satisfying conclusion.", 1),
    ("Despite some initially confusing plot elements, the film's emotional core shines through with remarkable clarity.", 1),
    ("The film's deliberate pacing might test some viewers' patience, but it builds to a climax that's absolutely worth the wait.", 1),
    ("While the film's character development is subtle, it creates a sense of authenticity that's rare in modern cinema.", 1),
    ("The film's minimalist approach to storytelling might seem sparse at first, but it creates space for powerful emotional moments.", 1),
    ("Though the film's themes are presented with some ambiguity, it invites meaningful discussion and interpretation.", 1),
    ("The film's unconventional camera work might be disorienting initially, but it creates a unique perspective on the story.", 1),
    ("While the film's score is occasionally unconventional, it perfectly captures the emotional journey of the characters.", 1),
    ("The film's experimental approach to genre might be challenging, but it results in a refreshingly original experience.", 1),
    ("Despite some initially confusing character motivations, the film's emotional payoff is deeply satisfying.", 1),
    ("The film's abstract visual elements might be puzzling at first, but they create a rich tapestry of meaning.", 1),
    ("While the film's dialogue style is deliberately stylized, it creates a unique and memorable atmosphere.", 1),
    ("The film's non-traditional structure might be challenging to follow, but it creates a uniquely engaging narrative experience.", 1),
    ("Though the film's pacing is deliberately measured, it allows for moments of profound emotional impact.", 1),
    ("The film's experimental use of sound might be unconventional, but it creates an immersive atmosphere.", 1),
    ("While the film's visual metaphors might be subtle, they add layers of meaning to the story.", 1),
    ("The film's complex character relationships might require careful attention, but they create a rich emotional tapestry.", 1),
    ("Despite some initially confusing plot elements, the film's emotional journey is deeply rewarding.", 1),
    ("The film's unconventional approach to storytelling might be challenging, but it creates a uniquely memorable experience.", 1),
    ("While the film's themes are presented with some ambiguity, they invite meaningful reflection and discussion.", 1),
    ("The film's experimental visual style might be polarizing, but it creates a distinctive and memorable atmosphere.", 1),
    ("A masterclass in visual storytelling that occasionally stumbles in its narrative pacing, but ultimately delivers a powerful emotional punch.", 1),
    ("The film's ambitious attempt to blend multiple genres creates some tonal inconsistencies, yet results in a uniquely engaging experience.", 1),
    ("While the film's complex plot structure might require multiple viewings to fully appreciate, each rewatch reveals new layers of meaning.", 1),
    ("The film's deliberate use of silence and space might test some viewers' patience, but it creates moments of profound emotional impact.", 1),
    ("Despite its unconventional approach to character development, the film manages to create deeply affecting relationships that stay with you.", 1),
    ("The film's experimental approach to sound design might be challenging at first, but it creates an immersive atmosphere that's unlike anything else.", 1),
    ("While the film's philosophical themes might seem heavy-handed at times, they're presented with such visual poetry that they become compelling.", 1),
    ("The film's non-linear narrative structure might be disorienting initially, but it rewards careful attention with a deeply satisfying resolution.", 1),
    ("Though the film's pacing is deliberately measured, it creates a hypnotic rhythm that draws you deeper into its world.", 1),
    ("The film's minimalist approach to dialogue might seem sparse, but it creates space for powerful visual storytelling.", 1),
    ("While the film's abstract visual elements might be puzzling at first, they create a rich tapestry of meaning that rewards interpretation.", 1),
    ("The film's ambitious attempt to tackle multiple themes occasionally leads to some narrative sprawl, but its emotional core remains strong.", 1),
    ("Despite some initially confusing character motivations, the film's emotional journey builds to a deeply satisfying conclusion.", 1),
    ("The film's experimental use of color and light might be unconventional, but it creates a visual language that's uniquely its own.", 1),
    ("While the film's complex character relationships might require careful attention, they create a rich emotional tapestry that's deeply rewarding.", 1),
    ("The film's unconventional approach to genre conventions might be challenging, but it results in a refreshingly original experience.", 1),
    ("Though the film's themes are presented with some ambiguity, they invite meaningful discussion and personal interpretation.", 1),
    ("The film's deliberate pacing might test some viewers' patience, but it creates a sense of immersion that's rare in modern cinema.", 1),
    ("While the film's visual metaphors might be subtle, they add layers of meaning that enrich the viewing experience.", 1),
    ("The film's experimental approach to storytelling might be polarizing, but it creates a uniquely memorable cinematic experience.", 1),
    ("Despite some initially confusing plot elements, the film's emotional journey builds to a powerful and satisfying conclusion.", 1),
    ("The film's unconventional use of music and sound might be challenging at first, but it creates an atmosphere that's uniquely compelling.", 1),
    ("While the film's character development is deliberately subtle, it creates a sense of authenticity that's deeply affecting.", 1),
    ("The film's ambitious visual style might be polarizing, but it creates a distinctive and memorable viewing experience.", 1),
    ("Though the film's narrative structure might be challenging to follow, it creates a uniquely engaging storytelling experience.", 1),
    ("The film's experimental approach to genre might be unconventional, but it results in a refreshingly original cinematic experience.", 1),
    ("While the film's themes are presented with some ambiguity, they invite meaningful reflection and personal interpretation.", 1),
    ("The film's deliberate use of space and silence might test some viewers' patience, but it creates moments of profound emotional impact.", 1),
    ("Despite its unconventional approach to storytelling, the film manages to create a deeply affecting and memorable experience.", 1),
    ("A complete waste of potential. The promising premise is squandered by poor execution, wooden acting, and a script that reads like a first draft.", 0),
    ("The film's stunning visuals are completely undermined by its nonsensical plot and laughable dialogue. A perfect example of style over substance.", 0),
    ("While the film's ambition is admirable, its execution is so flawed that it becomes frustrating to watch. A classic case of biting off more than it can chew.", 0),
    ("The film's attempts at emotional depth feel manipulative and unearned. Every emotional beat is telegraphed from miles away.", 0),
    ("A masterclass in how to waste a talented cast. The actors do their best, but they're given nothing to work with.", 0),
    ("The film's pacing is a complete mess - it drags through the middle and rushes the ending, making the whole experience unsatisfying.", 0),
    ("Despite its big budget, the film looks cheap and amateurish. The special effects are laughable, and the production design is uninspired.", 0),
    ("The film's attempts at being profound come across as pretentious and shallow. It's trying too hard to be something it's not.", 0),
    ("A beautiful exploration of human connection that occasionally stumbles in its pacing, but ultimately delivers a powerful emotional journey.", 1),
    ("The film's innovative approach to storytelling might be challenging at first, but it creates a uniquely engaging experience that rewards attention.", 1),
    ("While the film's complex themes might require careful consideration, they're presented with such visual poetry that they become compelling.", 1),
    ("The film's experimental use of sound and image creates an immersive atmosphere that's unlike anything else in contemporary cinema.", 1),
    ("A complete disaster from start to finish. The plot makes no sense, the characters are annoying, and the dialogue is painfully bad.", 0),
    ("The film's attempts at humor are cringeworthy and out of place. Every joke falls flat, making the experience even more painful.", 0),
    ("A refreshing take on a familiar genre that manages to find new emotional depths in well-trodden territory.", 1),
    ("The film's deliberate pacing might test some viewers' patience, but it creates moments of profound emotional impact.", 1),
    ("A perfect example of how not to make a movie. Every creative decision seems designed to frustrate and alienate the audience.", 0),
    ("The film's technical aspects are amateurish at best. Poor sound mixing, inconsistent lighting, and shaky camerawork throughout.", 0),
    ("A beautifully crafted character study that occasionally meanders, but always finds its way back to emotional truth.", 1),
    ("The film's ambitious scope occasionally leads to some narrative sprawl, but its emotional core remains strong and affecting.", 1),
    ("A complete waste of time and money. The film fails on every level - writing, direction, acting, and technical execution.", 0),
    ("The film's attempts at creating tension fall completely flat. Every 'twist' is painfully obvious from the start.", 0),
    ("A masterful blend of genres that creates a unique and memorable viewing experience, despite some pacing issues.", 1),
    ("The film's experimental approach to narrative structure might be challenging, but it results in a deeply rewarding experience.", 1),
    ("A perfect storm of bad filmmaking. Poor writing, terrible acting, and incompetent direction combine to create a truly awful experience.", 0),
    ("The film's attempts at social commentary are shallow and poorly integrated. It feels like an afterthought rather than a core element.", 0),
    ("A deeply affecting exploration of human relationships that occasionally stumbles in its execution, but always maintains its emotional core.", 1),
    ("The film's innovative visual style might be polarizing, but it creates a distinctive and memorable atmosphere that's uniquely its own.", 1),
    ("A complete failure of storytelling. The plot is incoherent, the characters are unlikable, and the ending is unsatisfying.", 0)
]

predicted_classes = []
predicted_probs = []
true_classes = []
probs_for_roc_pr = []

for text, true_label in sample_comments:
    predicted_class, predicted_probability, prob_positive = classify_sentiment(text, model, tokenizer, vocab, device)
    predicted_classes.append(predicted_class)
    predicted_probs.append(predicted_probability)
    true_classes.append(true_label)
    probs_for_roc_pr.append(prob_positive)
    print(f"Текст: {text}\nActual: {true_label}, Predicted: {predicted_class}, Probability: {predicted_probability}\n")

# Візуалізація результатів тестування
y_true = np.array(true_classes)
y_pred = np.array(predicted_classes)
y_probs = np.array(predicted_probs)
y_probs_roc_pr = np.array(probs_for_roc_pr)

# Створення DataFrame для скрипкового графіка
plot_data = pd.DataFrame({
    'Probability': y_probs,
    'True Label': ['Positive' if label == 1 else 'Negative' for label in y_true],
    'Predicted Label': ['Positive' if label == 1 else 'Negative' for label in y_pred]
})

# 1. Скрипковий графік
plt.figure(figsize=(10, 6))
sns.violinplot(data=plot_data, x='True Label', y='Probability', hue='Predicted Label',
               split=True, inner='quart', palette='Greens')
plt.title('Distribution of Prediction Probabilities by True and Predicted Labels')
plt.xlabel('True Label')
plt.ylabel('Prediction Probability')
plt.legend(title='Predicted Label')
plt.show()

# 2. ROC-крива
fpr, tpr, _ = roc_curve(y_true, y_probs_roc_pr)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='#207d3e', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# 3. Precision-Recall крива
precision, recall, _ = precision_recall_curve(y_true, y_probs_roc_pr)
average_precision = average_precision_score(y_true, y_probs_roc_pr)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='#207d3e', lw=2,
         label=f'PR curve (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.grid(True)
plt.show()

# 4. Теплова карта помилок
error_matrix = pd.crosstab(
    pd.Series(['Positive' if x == 1 else 'Negative' for x in y_true], name='True'),
    pd.Series(['Positive' if x == 1 else 'Negative' for x in y_pred], name='Predicted')
)

plt.figure(figsize=(8, 6))
sns.heatmap(error_matrix, annot=True, fmt='d', cmap='Greens')
plt.title('Error Distribution Heatmap')
plt.show()

# 5. Залежність точності від довжини тексту
text_lengths = [len(text.split()) for text, _ in sample_comments]
length_accuracy = pd.DataFrame({
    'Length': text_lengths,
    'Correct': y_true == y_pred
})

plt.figure(figsize=(8, 6))
sns.boxplot(data=length_accuracy, x='Correct', y='Length', palette=['#207d3e', '#00441b'])
plt.title('Text Length vs Prediction Accuracy')
plt.xlabel('Correct Prediction')
plt.ylabel('Text Length (words)')
plt.show()